In [ ]:
import tensorflow as tf
import numpy as np
import os
from preprocessing import getFilename,getData

#Config = tf.ConfigProto(device_count = {'GPU' : 0})

source_route = '/home/kexin/Documents/project/CASP_SCWRL'
tensorboard_route = '/home/kexin/Documents/project/model/train'
model_route = "/home/kexin/Documents/project/model/"

batch_size = 20
epoch = 15

x = tf.placeholder('float')
y = tf.placeholder('float')
test_pred = tf.placeholder('float')

train_filenames, train_binsizes = getFilename(source_route,400)

train_dataset = tf.data.Dataset.from_tensor_slices(train_filenames)
train_dataset = train_dataset.map( lambda text : tf.py_func(getData,[text], [tf.float64, tf.int32]))
train_dataset = train_dataset.repeat(epoch)
#train_dataset = train_dataset.batch(batch_size)

train_iterator = train_dataset.make_one_shot_iterator()
next_train_element = train_iterator.get_next()


test_filenames, test_binsizes = getFilename(source_route, 50)

test_dataset = tf.data.Dataset.from_tensor_slices(test_filenames)
test_dataset = test_dataset.map( lambda text : tf.py_func(getData,[text], [tf.float64, tf.int32]))
#test_dataset = test_dataset.batch(batch_size)

test_iterator = test_dataset.make_one_shot_iterator()
next_test_element = test_iterator.get_next()

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float64)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='VALID')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,3,3,3,1], strides=[1,2,2,2,1], padding='VALID')
    #Our output tensor produced by max_pooling2d() the 2x2 filter reduces height and width by 50% each.

def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,11,16])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,16,32])),
               'W_conv3':tf.Variable(tf.random_normal([3,3,3,32,32])),
               'W_conv4':tf.Variable(tf.random_normal([3,3,3,32,64])),
               'W_conv5':tf.Variable(tf.random_normal([3,3,3,64,128])),
               'W_conv6':tf.Variable(tf.random_normal([3,3,3,128,128])),
               'W_conv7':tf.Variable(tf.random_normal([3,3,3,128,256])),
               'W_conv8':tf.Variable(tf.random_normal([3,3,3,256,512])),
               #fc: first element is determined by result of layers below.
               'W_fc1':tf.Variable(tf.random_normal([512,256])),
               'W_fc2':tf.Variable(tf.random_normal([256,128])),
               'out':tf.Variable(tf.random_normal([128, 10]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([16])),
              'b_conv2':tf.Variable(tf.random_normal([32])),
              'b_conv3':tf.Variable(tf.random_normal([32])),
              'b_conv4':tf.Variable(tf.random_normal([64])),
              'b_conv5':tf.Variable(tf.random_normal([128])),
              'b_conv6':tf.Variable(tf.random_normal([128])),
              'b_conv7':tf.Variable(tf.random_normal([256])),
              'b_conv8':tf.Variable(tf.random_normal([512])),
              'b_fc1':tf.Variable(tf.random_normal([256])),
              'b_fc2':tf.Variable(tf.random_normal([128])),
              'out':tf.Variable(tf.random_normal([10]))}

    #The methods in the layers module for creating convolutional and pooling layers for two-dimensional 
    #image data expect input tensors to have a shape of [batch_size, image_height, image_width, channels] by default. 
    x = tf.reshape(x, shape=[-1, 120, 120, 120, 11])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = tf.layers.batch_normalization(conv2)
    conv2 = maxpool3d(conv2)
    
    conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = tf.layers.batch_normalization(conv3)

    conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = tf.layers.batch_normalization(conv4)
    conv4 = maxpool3d(conv4)
    
    conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    conv5 = tf.layers.batch_normalization(conv5)
    
    conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    conv6 = tf.layers.batch_normalization(conv6)

    conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    conv7 = tf.layers.batch_normalization(conv7)
    
    conv8 = tf.nn.relu(conv3d(conv7, weights['W_conv8']) + biases['b_conv8'])
    conv8 = tf.layers.batch_normalization(conv8)
    conv8 = maxpool3d(conv8)

    fc = tf.reshape(conv8,[-1, 1*1*1*512])
    
    fc1 = tf.nn.relu(tf.matmul(fc, weights['W_fc1'])+biases['b_fc1'])
    
    fc2 = tf.nn.relu(tf.matmul(fc1, weights['W_fc2'])+biases['b_fc2'])
    
    fc2 = tf.nn.dropout(fc2, keep_rate)

    output = tf.matmul(fc2, weights['out'])+biases['out']

    return output

def loss_function(prediction, y):
    
    hinge_loss = tf.reduce_mean(tf.losses.hinge_loss(labels=y, logits=prediction))
    
    tf.summary.scalar("hinge_loss", hinge_loss)
    
    return hinge_loss


def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    class_prediction = tf.nn.softmax(prediction)
    
    cost = loss_function(prediction, y)
    optimizer = tf.train.AdamOptimizer(learning_rate= 0.0001).minimize(cost)
    
    correct_pred = tf.equal(tf.argmax(test_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar("accuracy", accuracy)


    with tf.Session(config = Config) as sess:
        
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(tensorboard_route, sess.graph)
        sess.run(tf.global_variables_initializer())
        
        for ep in range(epoch):
            
            if epoch!=0 and epoch%2 == 0:
                
                saver = tf.train.Saver()
                save_route = model_route + str(ep) + ".ckpt" 
                saver.save(sess,save_route)
        

            epoch_loss = 0
            
            for binsize in train_binsizes:
                feature_array = []
                label_array = []
                
                pred_array = []
                try:
                    
                    bin_loss = 0
                    for bs in range(binsize):

                        feature, label = sess.run(next_train_element)
                        feature_array.append(feature)
                        label_array.append(label)
                    
                    #p = sess.run([prediction], feed_dict={x:feature_array})
                    #print ("predictions:", p)
                        #pred_array.append(sess.run(prediction, feed_dict={x: feature}))
                    
                    _, c = sess.run([optimizer, cost], feed_dict={x: feature_array, y: label_array})
                    
                    print("loss: ", c)
                    
                    epoch_loss += c

                except tf.errors.OutOfRangeError:
                    break

            print('Epoch', ep, 'completed out of',epoch,'loss:',epoch_loss)
            
            
            
            total_accuracy = 0
            
            for binsize in test_binsizes:
                
                test_x_array = []
                test_y_array = []
                
                test_pred_array = []
                
                try:
                    for bs in range(binsize):
                        test_x, test_y = sess.run(next_test_element)
                        test_x_array.append(test_x)
                        test_y_array.append(test_y)
                        
                        test_pred_array.append(sess.run(class_prediction, feed_dict={x: test_x}))
                        
                        a =  sess.run(accuracy, feed_dict={test_pred: test_pred_array, y:test_y_array})
                        total_accuracy += a
                        
                        print("batch Accuracy: ", a)
                    
                except tf.errors.OutOfRangeError:
                        break
                        
            print('Accuracy:', total_accuracy / len(test_binsizes))

train_neural_network(x)